In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [101]:
# Convert cartesian to orbital coordinates
# (r, v) = (x, y, z, v_x, v_y, v_z)
# (a, e, inclination, lon_asc_node, arg_peri, true_anom)

#set mu = 1
mu = 1

data = pd.read_csv("data/id000010-XV.csv")
# print(data.loc[0:5])
# print(data.iloc[:,0])

# Calculate h, r, and v magnitudes
h = 0
r = 0
v = 0

hx = data.iloc[:, 1] * data.iloc[:, 4] # x-component of the angular momentum
hy = data.iloc[:, 2] * data.iloc[:, 5] # y-component of the angular momentum
hz = data.iloc[:, 3] * data.iloc[:, 6] # z-component of the angular momentum

for i in range(1, 4):
    h += data.iloc[:,i] * data.iloc[:, i + 3]
    r += data.iloc[:,i]**2
    v += data.iloc[:,i + 3]**2

r = np.sqrt(r)
v = np.sqrt(v)

# print(h)
# print(r)
# print(v)

# Calculate E, a, e, inclination, lon_asc_node, arg_peri, and true_anom
E = 0
a = 0
e = 0
lon_asc_node = 0 
arg_peri = 0 
true_anom = 0

E = v**2 / 2 - mu / r
a = -1 * mu / (2 * E)
e = np.sqrt(1 - h**2 / (a * mu))

inclination = np.arccos(hz/h) 

lon_asc_node = np.arctan2(hx, -1 * hy)

true_anom = np.arcsin(a * (1 - e**2) / (h * e) * v) 

arg_peri = np.arcsin(data.iloc[:,3] / (r * np.sin(inclination))) - true_anom

# print(true_anom)
# print(arg_peri)

# Export to csv file
export_headers = ['t', 'a', 'e', 'inclination', 'lon_asc_node', 'arg_peri', 'true_anom']

export_data = pd.DataFrame(data.iloc[:,0])
# export_data[export_headers[0]] = data.iloc[:,0]
export_data[export_headers[1]] = a
export_data[export_headers[2]] = e
export_data[export_headers[3]] = inclination * 180 / np.pi
export_data[export_headers[4]] = lon_asc_node
export_data[export_headers[5]] = arg_peri * 180 / np.pi
export_data[export_headers[6]] = true_anom * 180 / np.pi

print(export_data)

export_data.to_csv("data/id000010-EL.csv", index = False, header = export_headers)


print("Done!")




                t          a         e  inclination  lon_asc_node   arg_peri  \
0             0.0  16.165177  0.999990    95.591649      1.862242   3.545822   
1        365250.0  17.005363  0.999985    88.889491      1.162648  -0.810074   
2        730500.0  17.792342  0.999983    84.738254      1.048460  -4.136291   
3       1095750.0  18.538816  0.999982    81.735819      1.007684  -6.889933   
4       1461000.0  19.250955  0.999981    79.753620      0.998411  -9.224614   
...           ...        ...       ...          ...           ...        ...   
996   363789000.0  19.439921  0.999985    86.590248     -2.650090  -2.615434   
997   364154250.0  20.220234  0.999986    82.729932     -2.569256  -5.503013   
998   364519500.0  21.068877  0.999988    78.852280     -2.530700  -8.508555   
999   364884750.0  21.920337  0.999990    75.480438     -2.518718 -11.479509   
1000  365250000.0  22.764899  0.999994    71.977854     -2.531025 -14.386910   

      true_anom  
0      0.003457  
1  

/class/datamine/apps/jupyter/kernels/lab/.venv/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/class/datamine/apps/jupyter/kernels/lab/.venv/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)
